In [2]:
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings(action='ignore')

In [3]:
train_df = pd.read_csv(".\\dataset\\train_data.csv")
test_df = pd.read_csv(".\\dataset\\test_data.csv")

df_lst = []
for i in range(8):
  df_lst.append(train_df[train_df.type == i])

In [4]:
for idx, df in enumerate(df_lst):
  print(f"type {idx} : {len(df)}")

type 0 : 432
type 1 : 369
type 2 : 366
type 3 : 306
type 4 : 306
type 5 : 249
type 6 : 249
type 7 : 186


In [5]:
# train_0 = train_df[train_df.type == 0]
# train_1 = train_df[train_df.type == 1]
# train_2 = train_df[train_df.type == 2]
# train_3 = train_df[train_df.type == 3]
# train_4 = train_df[train_df.type == 4]
train_5 = train_df[train_df.type == 5]
train_6 = train_df[train_df.type == 6]
train_7 = train_df[train_df.type == 7]

# train_0 = train_0.drop(columns=["out_pressure", "type"])
# train_1 = train_1.drop(columns=["out_pressure", "type"])
# train_2 = train_2.drop(columns=["out_pressure", "type"])
# train_3 = train_3.drop(columns=["out_pressure", "type"])
# train_4 = train_4.drop(columns=["out_pressure", "type"])
train_5 = train_5.drop(columns=["out_pressure", "type"])
train_6 = train_6.drop(columns=["out_pressure", "type"])
train_7 = train_7.drop(columns=["out_pressure", "type"])

In [6]:
# type 1에 비정상 처럼 보이는 부분도 샘플링 할 것
# train_1_1 = train_1[train_1.motor_vibe > 10]
# train_1_2 = train_1[train_1.motor_vibe < 10]

In [7]:
# print(len(train_1_1))
# print(len(train_1_2))

In [8]:
def generator_data(df: pd.DataFrame, _type: int, num: int):
  mu = df.mean()
  cov = df.corr()
  L = np.linalg.cholesky(cov)
  tmp_df = pd.DataFrame(columns=df.columns)
  for _ in range(num):
    while True:
      Z = np.random.normal(size=6)
      X = mu + np.dot(L, Z)
      if (X <= 0).sum() == 0:
        tmp_df.loc[len(tmp_df)] = X
        break
  df = pd.concat([df, tmp_df], ignore_index=True)
  df["out_pressure"] = [0.7 for _ in range(len(df))]
  df["type"] = [_type for _ in range(len(df))]
  return df

In [9]:
# train_0 = generator_data(train_0, 0, 70)
# train_1_1 = generator_data(train_1_1, 1, 50)
# train_1_2 = generator_data(train_1_2, 1, 80)
# train_2 = generator_data(train_2, 2, 130)
# train_3 = generator_data(train_3, 3, 200)
# train_4 = generator_data(train_4, 4, 200)
train_5 = generator_data(train_5, 5, 200)
train_6 = generator_data(train_6, 6, 200)
train_7 = generator_data(train_7, 7, 400)

df = pd.concat([test_df, train_5, train_6, train_7])
len(df)

8873

In [10]:
df.insert(2, "out_pressure", df.pop("out_pressure"))
df.head()

,air_inflow,air_end_temp,out_pressure,motor_current,motor_rpm,motor_temp,motor_vibe,type
0,2.51,53.28,0.7,32.54,2662.0,69.58,3.48,0
1,2.66,55.24,0.7,34.45,2819.0,71.32,3.57,0
2,1.72,42.74,0.7,22.23,1819.0,60.21,3.01,0
3,2.20,49.15,0.7,28.50,2332.0,65.91,3.30,0
4,2.06,47.28,0.7,26.67,2182.0,64.24,3.21,0


In [11]:
df.to_csv("test_ratio_fit.csv", index=False)

In [4]:
len(test_df)

7389